For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs.

Instructions
Open the categoricals variable we created before.
categoricals = data.select_dtypes(np.object)
categoricals.head()
Plot all the categorical variables with the proper plot. What can you see?
There might be some columns that seem to be redundant, check their values to be sure. What should we do with them?
Plot time variable. Can you extract something from it?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

In [3]:
df = pd.read_csv('lab-imbalanced-data-master/files_for_lab/customer_churn.csv')


In [4]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
columns = df.columns

In [7]:
for column in columns:
    print((column).upper())
    print(f'{df[column].value_counts()} \n')

CUSTOMERID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: customerID, Length: 7043, dtype: int64 

GENDER
Male      3555
Female    3488
Name: gender, dtype: int64 

SENIORCITIZEN
0    5901
1    1142
Name: SeniorCitizen, dtype: int64 

PARTNER
No     3641
Yes    3402
Name: Partner, dtype: int64 

DEPENDENTS
No     4933
Yes    2110
Name: Dependents, dtype: int64 

TENURE
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64 

PHONESERVICE
Yes    6361
No      682
Name: PhoneService, dtype: int64 

MULTIPLELINES
No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64 

INTERNETSERVICE
Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64 

ONLINESECURITY
No              

In [8]:
df = df.drop('customerID', axis=1)

In [9]:
# We will try to predict variable Churn using a logistic regression on variables tenure, SeniorCitizen, MonthlyCharges.

X = df[['SeniorCitizen', 'MonthlyCharges', 'tenure']]
y = df['Churn']

In [10]:
X.head(5)

,SeniorCitizen,MonthlyCharges,tenure
0,0,29.85,1
1,0,56.95,34
2,0,53.85,2
3,0,42.30,45
4,0,70.70,2


In [11]:
y.head(5)

0     No
1     No
2    Yes
3     No
4    Yes
Name: Churn, dtype: object

In [12]:
y.shape

(7043,)

In [13]:
y.value_counts(normalize=True)

No     0.73463
Yes    0.26537
Name: Churn, dtype: float64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
y_train.value_counts(normalize=True)

No     0.733582
Yes    0.266418
Name: Churn, dtype: float64

In [16]:
y_test.value_counts(normalize=True)

No     0.738822
Yes    0.261178
Name: Churn, dtype: float64

In [17]:
scaler = MinMaxScaler()

In [18]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
def logistic_regression_report(variable_data, target_data):
    '''Allows to run the Logistic Regression model and Classification Report for a given variable data and target data'''
    model = LogisticRegression()
    model.fit(variable_data, target_data)
    pred_train = model.predict(variable_data)
    pred_test = model.predict(X_test)
    print('TRAIN SET\nclassification report\n', classification_report(target_data, pred_train))
    print('TEST SET\nclassification report\n', classification_report(y_test, pred_test))

In [20]:
logistic_regression_report(X_train, y_train)

TRAIN SET
classification report
               precision    recall  f1-score   support

          No       0.82      0.91      0.87      4133
         Yes       0.66      0.46      0.54      1501

    accuracy                           0.79      5634
   macro avg       0.74      0.69      0.70      5634
weighted avg       0.78      0.79      0.78      5634

TEST SET
classification report
               precision    recall  f1-score   support

          No       0.82      0.90      0.86      1041
         Yes       0.62      0.46      0.53       368

    accuracy                           0.78      1409
   macro avg       0.72      0.68      0.69      1409
weighted avg       0.77      0.78      0.77      1409



In [21]:
sm = SMOTE(k_neighbors=3)
X_train_SMOTE, y_train_SMOTE = sm.fit_resample(X_train, y_train)

In [22]:
logistic_regression_report(X_train_SMOTE, y_train_SMOTE)

TRAIN SET
classification report
               precision    recall  f1-score   support

          No       0.73      0.73      0.73      4133
         Yes       0.73      0.73      0.73      4133

    accuracy                           0.73      8266
   macro avg       0.73      0.73      0.73      8266
weighted avg       0.73      0.73      0.73      8266

TEST SET
classification report
               precision    recall  f1-score   support

          No       0.88      0.72      0.79      1041
         Yes       0.47      0.72      0.57       368

    accuracy                           0.72      1409
   macro avg       0.68      0.72      0.68      1409
weighted avg       0.77      0.72      0.73      1409



In [23]:
tl = TomekLinks(sampling_strategy='all')
X_train_TL, y_train_TL = tl.fit_resample(X_train, y_train)

In [24]:
print('TRAIN')
print('X_train:', X_train.shape)
print('X_train_TL:',X_train_TL.shape

SyntaxError: unexpected EOF while parsing (1464779772.py, line 3)

In [25]:
y_train_TL.value_counts(normalize=True)

No     0.771005
Yes    0.228995
Name: Churn, dtype: float64

In [26]:
logistic_regression_report(X_train_TL, y_train_TL)

TRAIN SET
classification report
               precision    recall  f1-score   support

          No       0.87      0.93      0.90      3744
         Yes       0.70      0.52      0.60      1112

    accuracy                           0.84      4856
   macro avg       0.78      0.73      0.75      4856
weighted avg       0.83      0.84      0.83      4856

TEST SET
classification report
               precision    recall  f1-score   support

          No       0.83      0.90      0.86      1041
         Yes       0.63      0.46      0.53       368

    accuracy                           0.79      1409
   macro avg       0.73      0.68      0.70      1409
weighted avg       0.77      0.79      0.78      1409

